First we check all models for connectivity issues, and we make sure that there is a corresponding model file for each morphology. We create two directories with the clean models with and without axons, respectively. We also pre-calculate the morphology rotations for 10 deg steps, creating the corresponding file for each.

In [1]:
import numpy as np
import os, shutil

inhpath = '../Required_Files/Models/JSON Format/Inh Models/' # Location of Inhibitory json model files
excpath = '../Required_Files/Models/JSON Format/Exc Models/' # Location of Excitatory json model files
morpath = '../Required_Files/Models/SWCs transformed/' # Location of all swc morphology files

if os.path.exists('../Required_Files/Models/SWCs connected With'):  # Location for clean swc with axons
    shutil.rmtree('../Required_Files/Models/SWCs connected With')
os.mkdir('../Required_Files/Models/SWCs connected With/')
if os.path.exists('../Required_Files/Models/SWCs connected Without'):  # Location for clean swc without axons
    shutil.rmtree('../Required_Files/Models/SWCs connected Without')
os.mkdir('../Required_Files/Models/SWCs connected Without/')
if os.path.exists('../Required_Files/Models/SWCs connected Custom'):  # Location for clean swc with custom axons
    shutil.rmtree('../Required_Files/Models/SWCs connected Custom')
os.mkdir('../Required_Files/Models/SWCs connected Custom/')
if os.path.exists('../Required_Files/Models/SWCs not used'):  # Location for dropped swc files
    shutil.rmtree('../Required_Files/Models/SWCs not used')
os.mkdir('../Required_Files/Models/SWCs not used/')

inhtypes = os.listdir(inhpath)
exctypes = os.listdir(excpath)

allmodels = []
for i in inhtypes:
    for j in os.listdir(inhpath + i):
        allmodels.append(j)

for i in exctypes:
    for j in os.listdir(excpath + i):
        allmodels.append(j)

morphfiles = [s.replace('_transformed.swc','') for s in os.listdir(morpath)]
max_eucl_dendrite = 0
for i in morphfiles:
    print (i)
    if i not in allmodels: # Check if json file corresponding to current morphology exists
        shutil.move(morpath + i + '_transformed.swc', morpath.replace('SWCs transformed', 'SWCs not used') + \
                                                                                        i + '_transformed.swc')
    else:
        inFile = morpath + i + '_transformed.swc'
        outFileWith = morpath.replace('SWCs transformed', 'SWCs connected With') + i + '_transformed_0.swc'
        outFileWithout = morpath.replace('SWCs transformed', 'SWCs connected Without') + i + '_transformed_0.swc'
        outFileCustom = morpath.replace('SWCs transformed', 'SWCs connected Custom') + i + '_transformed_0.swc'
        fin = open(inFile, "rt")
        foutWith = open(outFileWith, "wt")
        foutWithout = open(outFileWithout, "wt")
        foutCustom = open(outFileCustom, "wt")
        soma_d = 1
        for line in fin:
                if line[0] == "#":
                    foutWith.write(line)
                    foutWithout.write(line)
                    foutCustom.write(line)
                else:
                    d,t,x,y,z,r,p = line.split(" ")
                    if int(t)==1:
                        soma_d = int(d)
                        xo=x
                        yo=float(y)
                        zo=z
                    if int(p)==-1 and int(t)!=1: # Check for dendrites without parents
                        line = d + " " + t + " " + x + " " + y + " " + z + " " + r + " " + str(soma_d) + "\n"
                        # print("Disconnect found " + i)
                    else:
                        line = d + " " + t + " " + x + " " + y + " " + z + " " + r + " " + p
                    foutWith.write(line)
                    foutWithout.write(line)
                    if int(t)!=2:
                        foutCustom.write(line)
        fin.close()
        foutWith.close()
        foutWithout.close()
        d = int(d)+1
        p = soma_d
        y = yo-20
        for axon_segment in range (100):
            line = str(int(d)) + " 2 " + xo + " " + str(y) + " " + zo + " 0.4 " + str(p)+ "\n"
            foutCustom.write(line)
            p = d
            d += 1
            y -= 20
        foutCustom.close()
        inFileWith = morpath.replace('SWCs transformed', 'SWCs connected With') + i + '_transformed_0.swc'
        inFileWithout = morpath.replace('SWCs transformed', 'SWCs connected Without') + i + '_transformed_0.swc'
        inFileCustom = morpath.replace('SWCs transformed', 'SWCs connected Custom') + i + '_transformed_0.swc'
        for j in range(10,361,10): # Calculate morphology rotations
            outFileWith = morpath.replace('SWCs transformed', 'SWCs connected With') + \
                                                    i + '_transformed_' + str(j) + '.swc'  
            outFileWithout = morpath.replace('SWCs transformed', 'SWCs connected Without') + \
                                                    i + '_transformed_' + str(j) + '.swc'   
            outFileCustom = morpath.replace('SWCs transformed', 'SWCs connected Custom') + \
                                                    i + '_transformed_' + str(j) + '.swc'    
            rotationXY = (np.pi/180)*j
            rotationYZ = (np.pi/180)*0 
            rotationXZ = (np.pi/180)*0
            
            finWith = open(inFileWith, "rt") # With axons
            foutWith = open(outFileWith, "wt")
            xo=0
            yo=0
            zo=0
            for line in finWith:
                    if line[0] == "#":
                        foutWith.write(line)
                    else:
                        d,t,x,y,z,r,p = line.split(" ")
                        if float(t)==1:
                            line = d+" "+t+" {0:.4f}".format(0)+" {0:.4f}".format(0)+" {0:.4f}".format(0)+" "+r+" "+p
                            foutWith.write(line)
                            xo=float(x)
                            yo=float(y)
                            zo=float(z)
                        else:
                            x=float(x)-xo
                            y=float(y)-yo
                            z=float(z)-zo
                            xn = x*np.cos(rotationXY)-y*np.sin(rotationXY)
                            y = x*np.sin(rotationXY)+y*np.cos(rotationXY)
                            x = xn
                            yn = y*np.cos(rotationYZ)-z*np.sin(rotationYZ)
                            z = y*np.sin(rotationYZ)+z*np.cos(rotationYZ)
                            y = yn
                            xn = x*np.cos(rotationXZ)-z*np.sin(rotationXZ)
                            z = x*np.sin(rotationXZ)+z*np.cos(rotationXZ)
                            x = xn
                            if float(t)==4:
                                line = d+" 3 {0:.4f}".format(x)+" {0:.4f}".format(y)+" {0:.4f}".format(z)+" "+r+" "+p
                            else:
                                line = d+" "+t+" {0:.4f}".format(x)+" {0:.4f}".format(y)+" {0:.4f}".format(z)+" "+r+" "+p
                            foutWith.write(line)
                            if (float(x)>max_eucl_dendrite):
                                max_eucl_dendrite = float(x)
                            if (float(y)>max_eucl_dendrite):
                                max_eucl_dendrite = float(y)
                            if (float(z)>max_eucl_dendrite):
                                max_eucl_dendrite = float(z)
            finWith.close()
            foutWith.close()
            
            finWithout = open(inFileWithout, "rt") # Without axons
            foutWithout = open(outFileWithout, "wt")
            xo=0
            yo=0
            zo=0
            for line in finWithout:
                    if line[0] == "#":
                        foutWithout.write(line)
                    else:
                        d,t,x,y,z,r,p = line.split(" ")
                        if float(t)==1:
                            line = d+" "+t+" {0:.4f}".format(0)+" {0:.4f}".format(0)+" {0:.4f}".format(0)+" "+r+" "+p
                            foutWithout.write(line)
                            xo=float(x)
                            yo=float(y)
                            zo=float(z)
                        else:
                            x=float(x)-xo
                            y=float(y)-yo
                            z=float(z)-zo
                            xn = x*np.cos(rotationXY)-y*np.sin(rotationXY)
                            y = x*np.sin(rotationXY)+y*np.cos(rotationXY)
                            x = xn
                            yn = y*np.cos(rotationYZ)-z*np.sin(rotationYZ)
                            z = y*np.sin(rotationYZ)+z*np.cos(rotationYZ)
                            y = yn
                            xn = x*np.cos(rotationXZ)-z*np.sin(rotationXZ)
                            z = x*np.sin(rotationXZ)+z*np.cos(rotationXZ)
                            x = xn
                            if float(t)==4:
                                line = d+" 3 {0:.4f}".format(x)+" {0:.4f}".format(y)+" {0:.4f}".format(z)+" "+r+" "+p
                            else:
                                line = d+" "+t+" {0:.4f}".format(x)+" {0:.4f}".format(y)+" {0:.4f}".format(z)+" "+r+" "+p
                            if float(t)!=2: # Ignore axons
                                foutWithout.write(line)
                            if (float(x)>max_eucl_dendrite):
                                max_eucl_dendrite = float(x)
                            if (float(y)>max_eucl_dendrite):
                                max_eucl_dendrite = float(y)
                            if (float(z)>max_eucl_dendrite):
                                max_eucl_dendrite = float(z)
            finWithout.close()
            foutWithout.close()
            
            finCustom = open(inFileCustom, "rt") # With custom axons
            foutCustom = open(outFileCustom, "wt")
            xo=0
            yo=0
            zo=0
            for line in finCustom:
                    if line[0] == "#":
                        foutCustom.write(line)
                    else:
                        d,t,x,y,z,r,p = line.split(" ")
                        if float(t)==1:
                            line = d+" "+t+" {0:.4f}".format(0)+" {0:.4f}".format(0)+" {0:.4f}".format(0)+" "+r+" "+p
                            foutCustom.write(line)
                            xo=float(x)
                            yo=float(y)
                            zo=float(z)
                        else:
                            x=float(x)-xo
                            y=float(y)-yo
                            z=float(z)-zo
                            xn = x*np.cos(rotationXY)-y*np.sin(rotationXY)
                            y = x*np.sin(rotationXY)+y*np.cos(rotationXY)
                            x = xn
                            yn = y*np.cos(rotationYZ)-z*np.sin(rotationYZ)
                            z = y*np.sin(rotationYZ)+z*np.cos(rotationYZ)
                            y = yn
                            xn = x*np.cos(rotationXZ)-z*np.sin(rotationXZ)
                            z = x*np.sin(rotationXZ)+z*np.cos(rotationXZ)
                            x = xn
                            if float(t)==4:
                                line = d+" 3 {0:.4f}".format(x)+" {0:.4f}".format(y)+" {0:.4f}".format(z)+" "+r+" "+p
                            else:
                                line = d+" "+t+" {0:.4f}".format(x)+" {0:.4f}".format(y)+" {0:.4f}".format(z)+" "+r+" "+p
                            foutCustom.write(line)
                            if (float(x)>max_eucl_dendrite):
                                max_eucl_dendrite = float(x)
                            if (float(y)>max_eucl_dendrite):
                                max_eucl_dendrite = float(y)
                            if (float(z)>max_eucl_dendrite):
                                max_eucl_dendrite = float(z)
            finCustom.close()
            foutCustom.close()
        os.remove(inFileWithout)
        outFile = morpath.replace('SWCs transformed', 'SWCs connected Without') + i + '_transformed_360.swc'  
        os.rename(outFile, inFileWithout)
        os.remove(inFileWith)
        outFile = morpath.replace('SWCs transformed', 'SWCs connected With') + i + '_transformed_360.swc'  
        os.rename(outFile, inFileWith)
        os.remove(inFileCustom)
        outFile = morpath.replace('SWCs transformed', 'SWCs connected Custom') + i + '_transformed_360.swc'  
        os.rename(outFile, inFileCustom)

ITL23__541557114
ITL23__569835804
ITL23__569860121
ITL23__571732727
ITL23__576110753
ITL23__576118161
ITL23__595572609
ITL23__595583062
ITL23__602165685
ITL23__653804581
ITL23__653854241
ITL23__654180841
ITL23__665713811
ITL23__677088033
ITL23__685806309
ITL23__685814252
ITL23__689300528
ITL23__689306818
ITL23__689309060
ITL23__689312221
ITL23__689323433
ITL23__689360253
ITL23__695494556
ITL23__695521538
ITL23__695546566
ITL23__695557339
ITL23__707828926
ITL23__716918890
ITL23__720850977
ITL23__720862326
ITL23__720871158
ITL23__720878801
ITL23__737089555
ITL23__737134157
ITL23__737158657
ITL23__737424303
ITL23__737820466
ITL23__737988336
ITL23__758285403
ITL23__758286194
ITL23__758311336
ITL23__758319694
ITL23__758323401
ITL23__758816993
ITL23__758967755
ITL23__767415421
ITL23__767433014
ITL23__767829778
ITL23__768819569
ITL23__768848167
ITL23__768867010
ITL23__768885440
ITL23__769228370
ITL23__770255641
ITL23__770268817
ITL23__770275864
ITL23__770307286
ITL23__770337451
ITL23__7703697

The used mechanisms should also be compiled for all future simulations.

In [2]:
from subprocess import call
import os

status = call("nrnivmodl modfiles", cwd = '../Required_Files/', shell=True)
if status:
    print ("NEURON ERROR")
else:
    print ("Compilation done!")

Compilation done!


Next, we manually choose 30 models, 5 from each sub-type, and calibrate them at the following target values.
We introduce noise in our DC injection, in order to achieve the desired deviation of the observed spike time per spike (Inter Spike Interval - ISI).

**cell_id** is the unique cell identification number <br />
**cell_type** is the sub-type of the cell <br />
**Target** is the target 1/ISI in Hz for this cell <br />
**TargetStD** is the desired deviation of 1/ISI <br />
**Mean** is the injected DC current in nA  (set at 0.1 nA initially, and it will be adjusted during calibration)<br />
**StD** is the deviation of the injected DC current in nA  (set at 0.01 nA initially, and it will be adjusted during calibration)<br />
**SM** is the DC current step for the calibration algorithm (set at 0.01 nA initially, and it will be adjusted during calibration) <br />
**SS** is the StD current step for the calibration algorithm (set at 0.001 nA initially, and it will be adjusted during calibration) <br />
**PM** is the last simulated 1/ISI result in Hz (fill with 0 for fresh start) <br />
**PS** is the last simulated StD of the 1/ISI in Hz (fill with 0 for fresh start) <br />
**Done** indicates the status of each cell and whether our target has been achieved or not (fill with 0 for fresh start) <br />

In [3]:
import pandas as pd

targets = pd.read_csv('../Required_Files/Calibrated.csv')
targets

,cell_id,cell_type,Target,TargetStD,Mean,StD,SM,SS,PM,PS,Done
0,595572609,ITL23,4.5,3.000000,0.246961,0.364400,7.812500e-06,1.000000e-03,4.519822,0.557221,0
1,911084481,ITL35,4.5,3.000000,0.059784,0.228513,7.376206e-17,5.506308e-14,4.667042,2.863592,0
2,1001462085,ITL46,4.5,3.000000,0.229678,0.396000,1.953125e-06,1.000000e-03,4.584236,0.406346,0
3,602165685,ITL23,7.0,4.666667,0.077025,0.008458,1.000000e-04,1.000000e-04,6.832788,5.056745,0
4,701072592,ITL35,7.0,4.666667,0.031746,0.419504,5.779534e-17,5.000000e-17,6.791843,2.925495,0
5,695533933,ITL46,7.0,4.666667,0.043433,0.593216,1.000000e-04,1.000000e-04,6.867787,4.300402,0
6,793490645,ITL23,8.0,5.333333,0.092120,0.765362,5.000000e-17,3.109454e-12,8.054360,5.795045,0
7,689331593,ITL35,8.0,5.333333,0.054029,0.035775,5.032315e-12,2.756995e-11,8.386013,5.437044,0
8,695564858,ITL46,8.0,5.333333,0.077209,0.138226,4.309533e-10,2.234062e-07,8.311416,5.087005,0
9,569860121,ITL23,28.0,18.666667,0.224194,0.045548,1.000000e-04,1.000000e-04,29.384793,17.879325,0


The calibration algorithm uses and refreshes the above table at each step. It sets up a new simulation directory for each cell, extracts the new ISI results, and refreshes all the necessary fields. We set it to run for 1000 steps, but it will stop if all cells have their **Done** value equal to 1.

In [1]:
import subprocess as sbp
import os, time, shutil, json
import pandas as pd

simulations_in_parallel = 4
current_vals = pd.read_csv('../Required_Files/Calibrated.csv').set_index('cell_id')
total_simulations = len(current_vals)

morpath = '../Required_Files/Models/SWCs connected Without/'
morphfiles = os.listdir(morpath)
morphfiles = [i for i in morphfiles if 'med_0.swc' in i]
morphfiles = [i for i in morphfiles if int(i.split('_')[2]) in current_vals.index]

inhpath = '../Required_Files/Models/JSON Format/Inh Models/'
excpath = '../Required_Files/Models/JSON Format//Exc Models/'
inhtypes = os.listdir(inhpath)
exctypes = os.listdir(excpath)

allmodels = []
for i in inhtypes:
    for j in os.listdir(inhpath + i):
        allmodels.append(inhpath + i + '/' + j + '/' + j + '_stage_1_hof_0.json')

for i in exctypes:
    for j in os.listdir(excpath + i):
        allmodels.append(excpath + i + '/' + j + '/' + j + '_stage_1_hof_0.json')

for stp in range(1000):
    print ("-----------------------------------------------Runnning STEP: "+str(stp))
    current_vals = pd.read_csv('../Required_Files/Calibrated.csv').set_index('cell_id')
    
    print ("Setting directories...")
    simulation_no = 0
    for i in morphfiles:
        base_path = os.getcwd().replace('\\','/') + '/Simulations/Calibrate/SIM_' + str(simulation_no)
        if os.path.exists(base_path):
            shutil.rmtree(base_path)
        os.makedirs(base_path)
        shutil.copyfile(morpath + i, base_path + '/morph.swc')
        for mod in allmodels:
            if i.split("transformed")[0] in mod:
                inpa = mod
                oupa = base_path + '/model.json'

                with open(inpa, 'r') as infile:
                    parameters = json.load(infile)

                json_object = json.dumps(parameters, indent=4)
                with open(oupa, 'w') as outfile:
                    outfile.write(json_object)
                break

            shutil.copyfile('../Required_Files/Simulate.py', base_path + '/simulate.py')
            shutil.copyfile('../Required_Files/cell_functions.py', base_path + '/cell_functions.py')
            shutil.copyfile('../Required_Files/nrnmech.dll', base_path + '/nrnmech.dll')

            inj_curr_val = current_vals.loc[int(i.split('_')[2]),'Mean']
            inj_std_val = current_vals.loc[int(i.split('_')[2]),'StD']
            done_val = current_vals.loc[int(i.split('_')[2]),'Done']
            fin = open(base_path + '/simulate.py', "rt")
            cnt = 10
            lines = []
            for line in fin:
                if cnt==10:
                    lines.append('base_dir = "' + base_path + '"\n')
                elif cnt==9:
                    lines.append('inj_curr = ' + str(inj_curr_val) + '\n')
                elif cnt==8:
                    lines.append('inj_std = ' + str(inj_std_val) + '\n')
                elif cnt==7:
                    lines.append('exp_rate = ' + str(40) + '\n')
                elif cnt==6:
                    lines.append('es_freq = ' + str(0) + '\n')
                elif cnt==5:
                    lines.append('es_ampl = ' + str(0) + '\n')
                elif cnt==4:
                    lines.append('mod_proc = "' + 'csmc_allactive' + '"\n')
                elif cnt==3:
                    lines.append('point = 0\n')
                elif cnt==2:
                    if done_val==1:
                        lines.append('run = 0\n')
                    else:
                        lines.append('run = 1\n')
                elif cnt==1:
                    lines.append('returns = 2\n')
                else:
                    lines.append(line)
                cnt -= 1
            fin.close()
            fout = open(base_path + '/simulate.py', "wt")
            for line in lines:
                fout.write(line)
            fout.close()
        simulation_no += 1

    print ("Simulating...")
    processes = []
    process_names = []
    active_count = 0
    process_index = 0
    
    start_time = time.time()
    while (active_count > 0) or (process_index < total_simulations):
        active_processes = processes.copy()
        active_process_names = process_names.copy()
        processes = []
        process_names = []
        for i, sbprc in enumerate(active_processes):
            poll = sbprc.poll()
            if poll is None:
                processes.append(sbprc)
                process_names.append(active_process_names[i])
            else:
                active_count -= 1
                print ('Finished ' + active_process_names[i])
                if process_index >= total_simulations:
                    print ('Active: ' + str(active_count) + ', Pending: ' + str(total_simulations-process_index))
        if (active_count < simulations_in_parallel) and (process_index < total_simulations):
            process_name = 'SIM_'+str(process_index)
            cmnd = ["python", os.getcwd().replace('\\','/') + '/Simulations/Calibrate/' + process_name + "/simulate.py"]
            p = sbp.Popen(cmnd)
            processes.append(p)
            process_names.append(process_name)
            active_count += 1
            process_index += 1
            print ('Active: ' + str(active_count) + ', Pending: ' + str(total_simulations-process_index))
        if active_count >= simulations_in_parallel:
            time.sleep(2)
        
    queue_time = time.time() - start_time
    print("Total time " + str(queue_time) + " seconds")
    
    print ("Refreshing...")
    resultsMean = []
    resultsStd = []
    initStep = 0
    alldone = 1
    simulation_no = 0
    for fil in morphfiles:
        i = int(fil.split('_')[2])
        if current_vals.loc[i,'Done']==1:
            simulation_no += 1
            continue
        base_path = os.getcwd().replace('\\','/') + '/Simulations/Calibrate/SIM_' + str(simulation_no) + \
                                                                        '/Simulation/output/Data.csv'
        df = pd.read_csv(base_path)
        if df.iloc[0,0]>1000:
            resultsMean.append(0)
        else:
            resultsMean.append(df.iloc[0,0])
        if df.iloc[1,0]>1000:
            resultsStd.append(0)
        else:
            resultsStd.append(df.iloc[1,0])  
        simulation_no += 1
        cond1 = (abs(resultsMean[-1]-current_vals.loc[i,'Target'])<current_vals.loc[i,'Target']/10 and \
                 abs(resultsStd[-1]-current_vals.loc[i,'TargetStD'])<current_vals.loc[i,'TargetStD'])
        cond2 = (current_vals.loc[i,'SM'] <= 0.00000000000000005) and \
                (current_vals.loc[i,'SS'] <= 0.00000000000000005)
        if cond1 or cond2:
            current_vals.loc[i,'Done'] = 1
            current_vals.loc[i,'PM'] = resultsMean[-1]
            current_vals.loc[i,'PS'] = resultsStd[-1]
            continue
        if initStep:
            if abs(resultsMean[-1]-current_vals.loc[i,'Target'])>current_vals.loc[i,'Target']/10:
                current_vals.loc[i,'SM'] = 0.001
            if abs(resultsStd[-1]-current_vals.loc[i,'TargetStD'])>current_vals.loc[i,'TargetStD']:
                current_vals.loc[i,'SS'] = 0.001
        if resultsMean[-1]>current_vals.loc[i,'Target']:
            if current_vals.loc[i,'PM']<current_vals.loc[i,'Target']:
                if current_vals.loc[i,'SM']>0.00000000000000005:
                    if abs(resultsMean[-1]-current_vals.loc[i,'Target'])>0.1:
                        current_vals.loc[i,'SM'] = current_vals.loc[i,'SM']*1.8
                    current_vals.loc[i,'SM'] = current_vals.loc[i,'SM']/2
                else:
                    current_vals.loc[i,'SM'] = 0.00000000000000005
                current_vals.loc[i,'Mean'] = current_vals.loc[i,'Mean'] - current_vals.loc[i,'SM']
            else:
                current_vals.loc[i,'Mean'] = current_vals.loc[i,'Mean'] - current_vals.loc[i,'SM']
        else:
            if current_vals.loc[i,'PM']>current_vals.loc[i,'Target']:
                if current_vals.loc[i,'SM']>0.00000000000000005:
                    current_vals.loc[i,'SM'] = current_vals.loc[i,'SM']/2
                else:
                    current_vals.loc[i,'SM'] = 0.00000000000000005
                current_vals.loc[i,'Mean'] = current_vals.loc[i,'Mean'] + current_vals.loc[i,'SM']
            else:
                current_vals.loc[i,'Mean'] = current_vals.loc[i,'Mean'] + current_vals.loc[i,'SM']
        current_vals.loc[i,'PM'] = resultsMean[-1]
        if current_vals.loc[i,'Mean']>4:
            current_vals.loc[i,'Mean'] = 0
            current_vals.loc[i,'SM'] = 0.001
        if resultsStd[-1]>current_vals.loc[i,'TargetStD']:
            if current_vals.loc[i,'PS']<current_vals.loc[i,'Target']:
                if current_vals.loc[i,'SS']>0.00000000000000005:
                    if abs(resultsStd[-1]-current_vals.loc[i,'TargetStD'])>0.1:
                        current_vals.loc[i,'SS'] = current_vals.loc[i,'SS']*1.8
                    current_vals.loc[i,'SS'] = current_vals.loc[i,'SS']/2
                else:
                    current_vals.loc[i,'SS'] = 0.00000000000000005
                current_vals.loc[i,'StD'] = current_vals.loc[i,'StD'] - current_vals.loc[i,'SS']
            else:
                current_vals.loc[i,'StD'] = current_vals.loc[i,'StD'] - current_vals.loc[i,'SS']
        else:
            if current_vals.loc[i,'PS']>current_vals.loc[i,'Target']:
                if current_vals.loc[i,'SS']>0.00000000000000005:
                    current_vals.loc[i,'SS'] = current_vals.loc[i,'SS']/2
                else:
                    current_vals.loc[i,'SS'] = 0.00000000000000005
                current_vals.loc[i,'StD'] = current_vals.loc[i,'StD'] + current_vals.loc[i,'SS']
            else:
                current_vals.loc[i,'StD'] = current_vals.loc[i,'StD'] + current_vals.loc[i,'SS']
        current_vals.loc[i,'PS'] = resultsStd[-1]
        if current_vals.loc[i,'StD']>4:
            current_vals.loc[i,'StD'] = 0
            current_vals.loc[i,'SS'] = 0.001
        elif current_vals.loc[i,'StD']<0:
            current_vals.loc[i,'StD'] = 0
            current_vals.loc[i,'SS'] = 0.001
        alldone = 0
    current_vals.to_csv('../Required_Files/Calibrated.csv')
    if alldone:
        print("All done..")
        break


-----------------------------------------------Runnning STEP: 0
Setting directories...
Simulating...
Active: 1, Pending: 29
Active: 2, Pending: 28
Active: 3, Pending: 27
Active: 4, Pending: 26
Finished SIM_0
Finished SIM_1
Finished SIM_3
Active: 2, Pending: 25
Active: 3, Pending: 24
Active: 4, Pending: 23
Finished SIM_4
Finished SIM_5
Finished SIM_6
Active: 2, Pending: 22
Active: 3, Pending: 21
Active: 4, Pending: 20
Finished SIM_7
Finished SIM_8
Finished SIM_9
Active: 2, Pending: 19
Active: 3, Pending: 18
Active: 4, Pending: 17
Finished SIM_10
Finished SIM_11
Finished SIM_12
Active: 2, Pending: 16
Active: 3, Pending: 15
Active: 4, Pending: 14
Finished SIM_13
Finished SIM_14
Finished SIM_15
Active: 2, Pending: 13
Active: 3, Pending: 12
Active: 4, Pending: 11
Finished SIM_16
Finished SIM_17
Finished SIM_18
Active: 2, Pending: 10
Active: 3, Pending: 9
Active: 4, Pending: 8
Finished SIM_19
Finished SIM_20
Finished SIM_21
Active: 2, Pending: 7
Active: 3, Pending: 6
Active: 4, Pending: 5
Fi

In [2]:
# See calibrated table
import pandas as pd

targets = pd.read_csv('../Required_Files/Calibrated.csv')
targets

,cell_id,cell_type,Target,TargetStD,Mean,StD,SM,SS,PM,PS,Done
0,595572609,ITL23,4.5,3.000000,0.246961,0.364400,7.812500e-06,1.000000e-03,4.482092,0.570550,1
1,911084481,ITL35,4.5,3.000000,0.059784,0.228513,7.376206e-17,5.506308e-14,4.297956,2.278800,1
2,1001462085,ITL46,4.5,3.000000,0.229678,0.396000,1.953125e-06,1.000000e-03,4.581481,0.407202,1
3,602165685,ITL23,7.0,4.666667,0.078895,0.517591,9.000000e-05,6.561000e-05,7.487165,7.150675,1
4,701072592,ITL35,7.0,4.666667,0.031746,0.419504,5.779534e-17,5.000000e-17,6.791843,2.925495,1
5,695533933,ITL46,7.0,4.666667,0.043433,0.593216,1.000000e-04,1.000000e-04,6.777218,4.263334,1
6,793490645,ITL23,8.0,5.333333,0.092120,0.765362,5.000000e-17,3.109454e-12,8.023477,5.791253,1
7,689331593,ITL35,8.0,5.333333,0.054029,0.035775,5.032315e-12,2.756995e-11,8.326112,5.340992,1
8,695564858,ITL46,8.0,5.333333,0.077209,0.138226,4.309533e-10,2.234062e-07,8.252609,4.999302,1
9,569860121,ITL23,28.0,18.666667,0.224194,0.045548,1.000000e-04,1.000000e-04,29.294764,17.674296,1


In [3]:
# Delete simulation directories
import shutil

base_path = '/Simulations/Calibrate/'
if os.path.exists(base_path):
    shutil.rmtree(base_path)

Then we pre-calculate the theoretical response for each cell versus its rotation, for different field source locations and field shapes (point and plane fields).

In [4]:
from neurom.core.morphology import iter_segments
from neurom.core.types import tree_type_checker as is_type
from neurom.core.dataformat import COLS
import pandas as pd
import neurom as nm
import numpy as np
import os

def perturb_estimate (file, distance_soma_field = [2500,0,0], PlaneField = False, neurite_type=nm.NeuriteType.all):
    col = 0
    morph = nm.load_morphology(file)
    segments = list(iter_segments(morph, neurite_filter=is_type(neurite_type)))
    if not segments:
        return np.nan
    segments = np.dstack(segments)
    seg_begs = segments[0, COLS.XYZ, :].T
    seg_ends = segments[1, COLS.XYZ, :].T
    seg_norm = seg_ends - seg_begs
    midpoints = 0.5 * (seg_begs + seg_ends)
    lengths = []
    for i in seg_norm:
        lengths.append(np.sqrt(i[0]**2 + i[1]**2 + i[2]**2))
    radii = np.mean(segments.T[:,-1],axis = 1)
    
    if PlaneField:
        distances = [distance_soma_field[0]-midpoints[:, col],
                     midpoints[:,1]-midpoints[:,1],
                     midpoints[:,1]-midpoints[:,1]]
        distancesstart = [distance_soma_field[0]-seg_begs[:, col],
                          seg_begs[:,1]-seg_begs[:,1],
                          seg_begs[:,1]-seg_begs[:,1]]
        distancesend = [distance_soma_field[0]-seg_ends[:, col],
                        seg_ends[:,1]-seg_ends[:,1],
                        seg_ends[:,1]-seg_ends[:,1]]
    else:
        distances = [distance_soma_field[0]-midpoints[:, 0],
                     distance_soma_field[1]-midpoints[:,1],
                     distance_soma_field[2]-midpoints[:,2]]
        distancesstart = [distance_soma_field[0]-seg_begs[:, 0],
                          distance_soma_field[1]-seg_begs[:,1],
                          distance_soma_field[2]-seg_begs[:,2]]
        distancesend = [distance_soma_field[0]-seg_ends[:, 0],
                        distance_soma_field[1]-seg_ends[:,1],
                        distance_soma_field[2]-seg_ends[:,2]]
    UPs = []
    Sides = []
    for i in range(len(distances[0])):
        a = np.array([distances[0][i], distances[1][i], distances[2][i]])
        b = np.array(seg_norm[i])
        startdist = np.linalg.norm([distancesstart[0][i], distancesstart[1][i], distancesstart[2][i]])
        enddist = np.linalg.norm([distancesend[0][i], distancesend[1][i], distancesend[2][i]])
        if startdist > enddist:
            UPs.append(np.linalg.norm((np.dot(b, a) / np.linalg.norm(a)**2 ) * a))
            Sides.append(np.linalg.norm(b-(np.dot(b, a) / np.linalg.norm(a)**2 ) * a))
        elif startdist < enddist:
            UPs.append(-np.linalg.norm((np.dot(b, a) / np.linalg.norm(a)**2 ) * a))
            Sides.append(np.linalg.norm(b-(np.dot(b, a) / np.linalg.norm(a)**2 ) * a))
        else:
            UPs.append(0)
            Sides.append(np.linalg.norm(b))
    cos8 = np.sin(np.arctan(np.array(UPs),np.array(Sides)))
    dircos8=[]
    for i in cos8:
        if i>=0:
            dircos8.append(1)
        else:
            dircos8.append(-1)
    return np.abs(np.nansum(radii*lengths*dircos8/np.abs(distances[0]**2 + distances[1]**2 + distances[2]**2)))

morpathWith = '../Required_Files/Models/SWCs connected With/'
morpathWithout = '../Required_Files/Models/SWCs connected Without/'
morphfilesWithAll = os.listdir(morpathWith)
morphfilesWithoutAll = os.listdir(morpathWithout)
morphfilesWith = morphfilesWithAll.copy()
morphfilesWithout = morphfilesWithoutAll.copy()

# # Uncomment to run only for calibrated cells
# short_list1 = list(pd.read_csv('../Required_Files/Calibrated.csv').loc[:,'cell_id'])
# short_list = []
# for i in short_list1:
#     short_list.append(int(i))
    
# morphfilesWith = []
# for i in morphfilesWithAll:
#     if int(i.split('_')[2]) in short_list:
#         morphfilesWith.append(i)
# print(len(morphfilesWith))

# morphfilesWithout = []
# for i in morphfilesWithoutAll:
#     if int(i.split('_')[2]) in short_list:
#         morphfilesWithout.append(i)
# print(len(morphfilesWithout))

# Without axon plane
results = []
results.append(['cell','rotation', 'estimate'])
for i in morphfilesWithout:
    results.append([i.split('_')[2],
                    i.split('_')[4].replace('.swc',''),
                    perturb_estimate (morpathWithout + i, PlaneField = True)])
df = pd.DataFrame(results[1:], columns = results[0])
df.to_csv('../Results/Result_Tables/PL_TH_NA_2500.csv', index = False)
# Without axon point 
for dist in [50, 100, 300, 500, 800, 1200, 1800, 2500]:
    results = []
    results.append(['cell','rotation', 'estimate'])
    for i in morphfilesWithout:
        results.append([i.split('_')[2],
                        i.split('_')[4].replace('.swc',''),
                        perturb_estimate (morpathWithout + i, distance_soma_field = [dist,0,0])])
    df = pd.DataFrame(results[1:], columns = results[0])
    df.to_csv('../Results/Result_Tables/PO_TH_NA_' + str(dist) + '.csv', index = False)
    
# With axon plane
results = []
results.append(['cell','rotation', 'estimate'])
for i in morphfilesWith:
    results.append([i.split('_')[2],
                    i.split('_')[4].replace('.swc',''),
                    perturb_estimate (morpathWith + i, PlaneField = True)])
df = pd.DataFrame(results[1:], columns = results[0])
df.to_csv('../Results/Result_Tables/PL_TH_WA_2500.csv', index = False)
# With axon point
#for dist in [50, 100, 300, 500, 800, 1200, 1800, 2500]:
#    results = []
#    results.append(['cell','rotation', 'estimate'])
#    for i in morphfilesWith:
#        results.append([i.split('_')[2],
#                        i.split('_')[4].replace('.swc',''),
#                        perturb_estimate (morpathWith + i, distance_soma_field = [dist,0,0])])
#    df = pd.DataFrame(results[1:], columns = results[0])
#    df.to_csv('../Results/Result_Tables/PO_TH_WA_' + str(dist) + '.csv', index = False)

C:\Users\Koza\AppData\Local\Programs\Python\Python310\lib\site-packages\h5py\__init__.py:85: H5pyDeprecationWarning: h5py.get_config().vds_min_hdf5_version is deprecated. This version of h5py does not support older HDF5 without VDS.
  if version.hdf5_version_tuple[:3] >= get_config().vds_min_hdf5_version:
